In [1]:
import os
import sys

home=os.path.expanduser("~")
os.environ["SPARK_HOME"]=os.path.join(home, 'spark-2.0.0-bin-hadoop2.7')
os.environ["PYLIB"]=os.path.join(os.environ["SPARK_HOME"],'python','lib')
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'py4j-0.10.1-src.zip'))
# python과 자바를 연결하는 파일을 경로 설정에 집어 넣는다. # pip install py4j
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'pyspark.zip'))
# pyspark.zip을 경로 설정에 집어 넣는다. # pip install pyspark

In [2]:
import pyspark
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder.master("local").appName("myApp").config(conf=myConf).getOrCreate()

# 실습 과제

In [3]:
_bicycle = spark.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('data/seoul_bicycle.csv')
bicycle=_bicycle\
    .withColumnRenamed("date", "Date")\
    .withColumnRenamed("count", "Count")


bicycle=bicycle.withColumn("year",bicycle.Date.substr(1, 4))
bicycle=bicycle.withColumn("month",bicycle.Date.substr(6, 2))

import pyspark.sql.functions as F

bicycle = bicycle\
    .withColumn('year', F.year('date'))\
    .withColumn('month', F.month('date'))

print ("==========================================================================================")
bicycle.groupBy('year').agg({"count":"sum"}).show()
print ("==========================================================================================")
print ("==========================================================================================")
bicycle.groupBy('year').pivot('month').agg({"count":"sum"}).show()
print ("==========================================================================================")

+----+----------+
|year|sum(count)|
+----+----------+
|2018|  10124874|
|2019|   1871935|
+----+----------+

+----+------+------+------+------+------+-------+-------+-------+-------+-------+------+------+
|year|     1|     2|     3|     4|     5|      6|      7|      8|      9|     10|    11|    12|
+----+------+------+------+------+------+-------+-------+-------+-------+-------+------+------+
|2018|164367|168741|462661|687885|965609|1207123|1100015|1037505|1447993|1420621|961532|500822|
|2019|495573|471543|904819|  null|  null|   null|   null|   null|   null|   null|  null|  null|
+----+------+------+------+------+------+-------+-------+-------+-------+-------+------+------+

